## Домашнее задание (часть 1)
1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать eda (как на занятии 6 с примером для оттока)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

* набор данных для бинарной классификации - данные о пассажирах Титаника https://www.kaggle.com/c/titanic/
* Целевым значением для нас будет значение `"Survived"` - то, выжил пассажир или нет.

In [35]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score,recall_score

In [2]:
data = pd.read_csv("titanic_train.csv")
#target_col = data["Survived"]
#Id_col = data['PassengerId']
target = 'Survived'
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
len(data) - data.count()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Data preprocessing

In [4]:
data.drop(["Cabin", "Name", "Ticket"], axis=1, inplace=True)

In [5]:
data["Age"].fillna(data["Age"].mean(), inplace=True)

In [6]:
data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [7]:
data["Embarked"].fillna("S", inplace=True)

In [8]:
len(data) - data.count()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 62.7+ KB


In [10]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


### EDA

In [11]:
#labels
lab = data[target].value_counts().keys().tolist()
#values
val = data[target].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Customer attrition in data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data1 = [trace]
fig = go.Figure(data = data1,layout = layout)
py.iplot(fig)

In [12]:
#Separating churn and non churn customers
Survived     = data[data["Survived"] == 1]
Not_Survived = data[data["Survived"] == 0]

#Separating catagorical and numerical columns
Id_col     = ['PassengerId']
target_col = ["Survived"]
cat_cols   = data.nunique()[data.nunique() < 6].keys().tolist()
num_cols   = [x for x in data.columns if x not in cat_cols + target_col + Id_col]

#function  for pie plot for customer attrition types
def plot_pie(column) :
    
    trace1 = go.Pie(values  = Survived[column].value_counts().values.tolist(),
                    labels  = Survived[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    domain  = dict(x = [0,.48]),
                    name    = "Survived passengers",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    hole    = .6
                   )
    trace2 = go.Pie(values  = Not_Survived[column].value_counts().values.tolist(),
                    labels  = Not_Survived[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    domain  = dict(x = [.52,1]),
                    hole    = .6,
                    name    = "Non survived Passengers" 
                   )


    layout = go.Layout(dict(title = column + " distribution passengers ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            annotations = [dict(text = "Survived passengers",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .15, y = .5),
                                           dict(text = "Non survived Passengers",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .88,y = .5
                                               )
                                          ]
                           )
                      )
    data1 = [trace1,trace2]
    fig  = go.Figure(data = data1,layout = layout)
    py.iplot(fig)


#function  for histogram for customer attrition types
def histogram(column) :
    trace1 = go.Histogram(x  = Survived[column],
                          histnorm= "percent",
                          name = "Survived passengers",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = .9 
                         ) 
    
    trace2 = go.Histogram(x  = Not_Survived[column],
                          histnorm = "percent",
                          name = "Non survived Passengers",
                          marker = dict(line = dict(width = .5,
                                              color = "black"
                                             )
                                 ),
                          opacity = .9
                         )
    
    data1 = [trace1,trace2]
    layout = go.Layout(dict(title =column + " distribution passengers ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = column,
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = "percent",
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                           )
                      )
    fig  = go.Figure(data=data1,layout=layout)
    
    py.iplot(fig)
    


#for all categorical columns plot pie
for i in cat_cols :
    plot_pie(i)

#for all categorical columns plot histogram    
for i in num_cols :
    histogram(i)



In [13]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [14]:
bin_cols   = data.nunique()[data.nunique() == 2].keys().tolist()

#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    data[i] = le.fit_transform(data[i])
    

In [15]:
data["Pclass"] = data["Pclass"].astype("category")
data = pd.get_dummies(data)

In [16]:
data.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,1,0,1,22.0,1,0,7.2500,0,0,1,0,0,1
1,2,1,0,38.0,1,0,71.2833,1,0,0,1,0,0
2,3,1,0,26.0,0,0,7.9250,0,0,1,0,0,1
3,4,1,0,35.0,1,0,53.1000,1,0,0,0,0,1
4,5,0,1,35.0,0,0,8.0500,0,0,1,0,0,1


In [17]:
summary = (data[[i for i in data.columns 
                 #if i not in Id_col
                ]].
           describe().transpose().reset_index())

summary = summary.rename(columns = {"index" : "feature"})
summary = np.around(summary,3)

val_lst = [summary['feature'], summary['count'],
           summary['mean'],summary['std'],
           summary['min'], summary['25%'],
           summary['50%'], summary['75%'], summary['max']]

trace  = go.Table(header = dict(values = summary.columns.tolist(),
                                line = dict(color = ['#506784']),
                                fill = dict(color = ['#119DFF']),
                               ),
                  cells  = dict(values = val_lst,
                                line = dict(color = ['#506784']),
                                fill = dict(color = ["lightgrey",'#F5F8FF'])
                               ),
                  columnwidth = [200,60,100,100,60,60,80,80,80])
layout = go.Layout(dict(title = "Variable Summary"))
figure = go.Figure(data=[trace],layout=layout)
py.iplot(figure)

In [18]:
#correlation
correlation = data.corr()
#tick labels
matrix_cols = correlation.columns.tolist()
#convert to array
corr_array  = np.array(correlation)

#Plotting
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data1 = [trace]
fig = go.Figure(data=data1,layout=layout)
py.iplot(fig)

In [19]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
#PassengerId col
Id_col     = ['PassengerId']
#Target columns
target_col = ["Survived"]

X = data[[i for i in data.columns if i not in Id_col + target_col]]
#Y = target_col + Id_col
Y = data[target_col + Id_col]

principal_components = pca.fit_transform(X)
pca_data = pd.DataFrame(principal_components,columns = ["PC1","PC2"])
pca_data = pca_data.merge(Y,left_index=True,right_index=True,how="left")
#pca_data["Survived"] = pca_data["Survived"].replace({1:"Churn",0:"Not Churn"})

def pca_scatter(target,color) :
    tracer = go.Scatter(x = pca_data[pca_data["Survived"] == target]["PC1"] ,
                        y = pca_data[pca_data["Survived"] == target]["PC2"],
                        name = target,mode = "markers",
                        marker = dict(color = color,
                                      line = dict(width = .5),
                                      symbol =  "diamond-open"),
                        #text = ("Passenger Id : " + 
                        #        pca_data[pca_data["Survived"] == target]['PassengerId'])
                       )
    return tracer

layout = go.Layout(dict(title = "Visualising data with principal components",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 1",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 2",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        height = 600
                       )
                  )
trace1 = pca_scatter(1,'red')
trace2 = pca_scatter(0,'royalblue')
data1 = [trace2,trace1]
fig = go.Figure(data=data1,layout=layout)
py.iplot(fig)

In [57]:
y = data[target]
X = data.drop(target, axis=1)

In [58]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [59]:
lr = LogisticRegression()

lr.fit(x_train, y_train)
y_predict = lr.predict(x_test)

C:\Users\sych_\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [60]:
def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 72.62%
roc: 77.55%
recall: 68.54%
precision: 77.22%


### Test the PU learning approach

* Оставим только 50 процентов позитивов

In [41]:
data[target].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [43]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data[target].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 50% of the positives marked
pos_sample_len = int(np.ceil(0.5 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 171/342 as positives and unlabeling the rest


* Create the target col 'class_test' that will be 1 for postive and -1 for unlabebed

In [45]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    720
 1    171
Name: class_test, dtype: int64


In [46]:
mod_data.head(10)

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,class_test
0,1,0,1,22.000000,1,0,7.2500,0,0,1,0,0,1,-1
1,2,1,0,38.000000,1,0,71.2833,1,0,0,1,0,0,1
2,3,1,0,26.000000,0,0,7.9250,0,0,1,0,0,1,1
3,4,1,0,35.000000,1,0,53.1000,1,0,0,0,0,1,-1
4,5,0,1,35.000000,0,0,8.0500,0,0,1,0,0,1,-1
5,6,0,1,29.699118,0,0,8.4583,0,0,1,0,1,0,-1
6,7,0,1,54.000000,0,0,51.8625,1,0,0,0,0,1,-1
7,8,0,1,2.000000,3,1,21.0750,0,0,1,0,0,1,-1
8,9,1,0,27.000000,0,2,11.1333,0,0,1,0,0,1,1
9,10,1,0,14.000000,1,0,30.0708,0,1,0,1,0,0,-1


### random negative sampling

In [97]:
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
neg_sample.drop(target, axis=1, inplace=True)
neg_sample['class_test'] = 0
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
sample_test.drop("class_test", axis=1, inplace=True)
cols = sample_test.columns.tolist()
cols = [cols[0]] + cols[2:] + [cols[1]]
sample_test = sample_test[cols]
pos_sample = mod_data[mod_data['class_test']==1]
pos_sample.drop(target, axis=1, inplace=True)
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(171, 13) (171, 13)


In [98]:
sample_train.head()

,PassengerId,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,class_test
79,80,0,30.0,0,0,12.4750,0,0,1,0,0,1,1
810,811,1,26.0,0,0,7.8875,0,0,1,0,0,1,0
520,521,0,30.0,0,0,93.5000,1,0,0,0,0,1,1
230,231,0,35.0,1,0,83.4750,1,0,0,0,0,1,1
844,845,1,17.0,0,0,8.6625,0,0,1,0,0,1,0


In [99]:
sample_test.head()

,PassengerId,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Survived
259,260,0,50.0,0,1,26.0000,0,1,0,0,0,1,1
795,796,1,39.0,0,0,13.0000,0,1,0,0,0,1,0
588,589,1,22.0,0,0,8.0500,0,0,1,0,0,1,0
310,311,0,24.0,0,0,83.1583,1,0,0,1,0,0,1
779,780,0,43.0,0,1,211.3375,1,0,0,0,0,1,1


In [104]:
sample_test.shape #все неразмеченные пассажиры, будем их предсказывать и ставнивать с исходным таргетом

(549, 13)

In [100]:
model = LogisticRegression()

model.fit(sample_train.iloc[:,:-1].values, 
          sample_train.iloc[:,-1].values)
y_predict = model.predict(sample_test.iloc[:,:-1].values)
evaluate_results(sample_test[target].values, y_predict)

Classification results:
f1: 60.07%
roc: 74.73%
recall: 65.89%
precision: 55.19%


C:\Users\sych_\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Видно, что качество стало хуже, но в целом неплохое